# Генератор облаков слов по продуктам для отзывов Amazon

В этом проекте вы создадите **генератор облаков слов по продуктам** с использованием данных отзывов Amazon. Этот генератор будет создавать облака слов на основе наиболее частых слов в отзывах клиентов для каждого уникального продукта. Каждое облако слов будет показывать до 50 самых распространенных слов для продукта, предоставляя представление о том, как клиенты описывают и воспринимают его.

### Цели проекта

Завершив этот проект, вы:
- **Загрузите и исследуете набор данных** с использованием библиотеки `datasets`.
- **Предобработаете текстовые данные**, преобразовав их в нижний регистр и удалив пунктуацию.
- **Удалите стоп-слова**, чтобы оставить только значимые слова.
- **Сгруппируете отзывы по продуктам** с использованием поля `parent_asin`.
- **Сгенерируете облака слов** с наиболее частыми словами для каждого продукта.

Вы можете прочитать больше о наборе данных [здесь на Hugging Face](https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023).

---

### Информация о наборе данных

Мы будем использовать набор данных **Amazon Reviews 2023**, который содержит отзывы с Amazon по различным категориям продуктов. Каждый отзыв включает такие поля, как рейтинг, заголовок, текст отзыва, изображения, идентификатор продукта (`asin`), `parent_asin` (идентификатор группы продуктов) и другие детали.

#### Релевантные поля для этого проекта
- **parent_asin**: Уникальный идентификатор для каждой группы продуктов.
- **text**: Тело отзыва клиента.
- **title**: Заголовок отзыва.

Каждый отзыв предоставляет информацию о продукте. Анализируя эти отзывы, мы можем определить общие слова, связанные с каждой группой продуктов.

---

### Пошаговое руководство

Следуйте этим шагам, чтобы завершить проект:

---

### Шаг 0: Установите необходимые библиотеки



In [ ]:
%pip install datasets


### Шаг 1: Загрузите набор данных отзывов Amazon

Сначала загрузите набор данных отзывов Amazon с Hugging Face, используя библиотеку `datasets`. Мы выведем первую запись, чтобы понять структуру данных.

#### Задача 1: Загрузите и изучите набор данных.



In [ ]:
from datasets import load_dataset,Dataset

def load_amazon_reviews() -> Dataset:
    """
    Загружает набор данных отзывов Amazon с Hugging Face.

    Возвращает:
        dataset (Dataset): Полный набор данных, загруженный с Hugging Face.
    """
    dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)
    return dataset

# Загрузите набор данных и выведите пример отзыва
dataset = load_amazon_reviews()


In [ ]:
print(dataset["full"][0])


Шаг 2: Предобработка текста

Чтобы подготовить текст отзыва для анализа, преобразуйте его в нижний регистр и удалите любую пунктуацию. Это стандартизирует текстовые данные, позволяя лучше анализировать частоту.

Задача 2: Реализуйте функцию preprocess_text для очистки текста.



In [ ]:
import string

def preprocess_text(text) -> str:
  text=text.lower()
  text=text.translate(str.maketrans("","",string.punctuation))
  return text

# Пример использования
sample_text = dataset["full"][0]["text"]
processed_text = preprocess_text(sample_text)
print("Обработанный текст:", processed_text)


Шаг 3: Удаление стоп-слов

Стоп-слова — это общие слова (например, "the", "is" и "and"), которые обычно не несут значительного смысла. Их удаление помогает сосредоточиться на более значимых словах.

Задача 3: Реализуйте функцию remove_stop_words для фильтрации стоп-слов.



In [ ]:
%pip install nltk

In [7]:
import nltk
from nltk.corpus import stopwords

In [ ]:


# Загрузите набор данных стоп-слов, если он еще не загружен
nltk.download("stopwords")

# Загрузите английские стоп-слова
stop_words = set(stopwords.words("english"))

def remove_stop_words(text) -> list:
  words=text.split() # Разделяет на отдельные слова
  filtered=[word for word in words if word.lower() not in stop_words ]# удаляет стоп-слова
  return filtered

# Пример использования
filtered_words = remove_stop_words(processed_text)
print("Отфильтрованные слова:", filtered_words)


Шаг 4: Группировка отзывов по parent_asin

Сгруппируйте все тексты отзывов по продукту (parent_asin), чтобы у каждого продукта был объединенный текст всех его отзывов. Это позволяет анализировать частоту слов каждого продукта в комплексной форме.

Задача 4: Реализуйте функцию group_reviews_by_product для организации отзывов по продуктам.



In [9]:
from collections import defaultdict
from tqdm import tqdm

In [ ]:
def group_reviews_by_product(dataset) -> dict:
  product_reviews=defaultdict(list) # Создает пустой словарь
  for item in dataset["full"]:
    product_reviews[item["parent_asin"]].append(item["text"]) # Текст отзыва в список отзывов соответсвующий parent_asin
  for parent_asin,reviews in product_reviews.items():
    print (f"{parent_asin}: {reviews}")  # Выводим parent_asin и соответствующий ему отзывы
  return product_reviews
# Пример использования
product_reviews = group_reviews_by_product(dataset)
print("Пример объединенного текста отзыва для продукта:", list(product_reviews.values())[0]) # Выводдим список отзывов ля первого продукта


Шаг 5: Генерация облака слов (топ 50 слов) для каждого продукта

Теперь мы подсчитаем частоту слов для каждого продукта и оставим только топ 50 слов по частоте.

Задача 5: Реализуйте функцию generate_word_cloud для возврата топ 50 слов по частоте.



In [47]:
from collections import Counter
# Определите глобальный параметр для топ K слов
TOP_K = 50
def generate_word_cloud(words: list, top_k=TOP_K) -> dict:
  word_counts = defaultdict(int)
  for word in words:
        word_counts[word] += 1
  sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
  return dict(sorted_words[:top_k])


Шаг 6: Объедините все для генерации облаков слов по продуктам

Наконец, объедините функции для генерации облаков слов для каждого продукта в наборе данных.

Задача 6: Используйте generate_product_word_clouds для создания облаков слов по продуктам.



In [ ]:
from collections import defaultdict,Counter
from tqdm import tqdm
def generate_product_word_clouds(dataset, top_k=TOP_K) -> dict:
  word_clouds={}
  for parent_asin,reviews in product_reviews.items(): # Проходим по каждому продукту и его отзыву
    all_reviews=[] # Пустой список
    for review in reviews:
      processed_words=preprocess_text(review)
      filltered_words=remove_stop_words(processed_words)
    all_reviews. extend(filltered_words)
    word_clouds[parent_asin]=generate_word_cloud(all_reviews,top_k) # Добавляем отфилтрованые слова в общий список слов
  return word_clouds # Ненерируем облако слов

    # Сгенерируйте облака слов и выведите пример результата
product_word_clouds = generate_product_word_clouds(dataset)
sample_asin = list(product_word_clouds.keys())[0]
print(f"Топ слов для продукта {sample_asin}:")
print(product_word_clouds[sample_asin])



---
### Резюме

В этом проекте вы завершили генератор облаков слов по продуктам, выполнив:

1.	Загрузку набора данных отзывов Amazon.
2.	Предобработку и очистку текстовых данных.
3.	Удаление стоп-слов.
4.	Группировку отзывов по parent_asin.
5.	Генерацию облаков слов с топ 50 наиболее частыми словами для каждого продукта.

Это предоставляет представление о мнениях клиентов о каждом продукте, демонстрируя слова, наиболее ассоциируемые с каждым товаром.
